In [1]:
# Get Crime Incidents February 2017 to 5th July 2023 for PG county
import requests
import json
import pandas as pd
import sqlite3


offset = 0
limit = 10000
full_historical_crime_data = []

while True:
  url = f'https://data.princegeorgescountymd.gov/resource/wb4e-w4nf.json?$offset={offset}&$limit={limit}'
  x = requests.get(url)
  nested = json.loads(json.dumps(x.json(), indent=2))
  if len(nested) == 0:
    break
  full_historical_crime_data = full_historical_crime_data + nested
  offset = offset + limit

print("Number of Crime Incidents February 2017 to 5th July 2023:", len(full_historical_crime_data))
historical_df = pd.json_normalize(full_historical_crime_data)

historical_df = historical_df[historical_df['location.latitude'].notna()]
historical_df = historical_df[historical_df['location.longitude'].notna()]
historical_df = historical_df.drop(columns=['location.latitude', 'location.longitude', 'location.human_address', ':@computed_region_87xh_ddyp'])

historical_df.head()

Number of Crime Incidents February 2017 to 5th July 2023: 150608


,incident_case_id,date,clearance_code_inc_type,pgpd_reporting_area,pgpd_sector,pgpd_beat,street_address,latitude,longitude,street_number
0,PP17020200002452,2017-02-02T00:00:00.000,ACCIDENT,826,D,D1,96TH AVE / GOOD LUCK RD,38.98081897199154,-76.84693802893162,NaN
1,PP17020300000263,2017-02-03T00:00:00.000,ACCIDENT,602,J,J4,INDIAN HEAD HWY SB / WILSON BRIDGE DR,38.78724980354309,-76.99851253628731,NaN
2,PP17020300001861,2017-02-03T00:00:00.000,THEFT,405,H,H2,1484 ADDISON RD S,38.86697405576706,-76.89221352338791,1400 BLOCK
3,PP17020300000798,2017-02-03T00:00:00.000,THEFT FROM AUTO,219,B,B63,4500 COLLEGE AVE,38.98213131725788,-76.93781711161137,4500 BLOCK
4,PP17020300001916,2017-02-03T00:00:00.000,ACCIDENT,423,H,H7,5510 SILVER HILL RD SB,38.85286647081375,-76.91321086883545,5500 BLOCK


In [ ]:
conn = sqlite3.connect('historical_crime_data.db')
historical_df.to_sql('crimes', conn, if_exists='replace')